In [1]:
import os
import sys
import tempfile
import pandas as pd

from credential.yamlhandler import YAMLHandler
from vbout.vboutapi import VboutApi

In [2]:
sys.path.append('/Users/jeanmermozeffi/DataspellProjects/scraping-master/module')

## Parameters

In [3]:
file_path = 'credential/secrets.yml'
handler = YAMLHandler(file_path)
keys, links, path = handler.get_security_elements('security')

In [4]:
API_KEY = keys.get('API_KEY_VBOUT')
LIMIT = 5000
CREDENTIALS_PATH = "/Users/jeanmermozeffi/.google/credentials/secret-reporting-sheet.json"
SPREADSHEET_NAME = 'Repport Mailling Data'
NEW_SHEET_NAME = 'Campagnes Mailing'
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = CREDENTIALS_PATH
GOOGLE_APPLICATION_CREDENTIALS = os.getenv("GOOGLE_APPLICATION_CREDENTIALS")

In [5]:
print(API_KEY)

6105505270701001086023538


## Test API

In [11]:
from_date, to_date = vbout.previous_month_dates()
from_date, to_date

('03/01/2024', '03/31/2024')

In [6]:
vbout = VboutApi()

In [15]:
from_to = '01/01/2024'
to_end = '01/31/2024'
campaign_data_list, from_date, to_date = vbout.get_campaigns_data(from_date=from_to, to_date=to_end)
campaign_data_list

Aucune campagne trouvée.


TypeError: cannot unpack non-iterable NoneType object

In [14]:
campaign_data_list[0]['id']

'196728'

In [11]:
if campaign_data_list:
    campaign_ids = list(map(lambda entry: entry['id'], campaign_data_list))
    campaign_titles = vbout.get_campaign_titles(campaign_ids)
    campaigns_stats, total_all_clicks_stats, total_unique_clicks_stats, errors_campaigns = vbout.get_campaigns_stats(campaign_ids)
    lists = [campaign_data_list, campaign_titles, campaigns_stats, total_all_clicks_stats, total_unique_clicks_stats]

In [12]:
campaigns_stats

[{'open': 245, 'unopened': 912, 'bounced': 30, 'unsubscribed': 0},
 {'open': 421, 'unopened': 746, 'bounced': 28, 'unsubscribed': 0},
 {'open': 238, 'unopened': 937, 'bounced': 27, 'unsubscribed': 1},
 {'open': 235, 'unopened': 947, 'bounced': 30, 'unsubscribed': 1},
 {'open': 223, 'unopened': 968, 'bounced': 31, 'unsubscribed': 1},
 {'open': 244, 'unopened': 960, 'bounced': 24, 'unsubscribed': 1},
 {'open': 247, 'unopened': 963, 'bounced': 28, 'unsubscribed': 0},
 {'open': 307, 'unopened': 907, 'bounced': 34, 'unsubscribed': 1},
 {'open': 366, 'unopened': 854, 'bounced': 40, 'unsubscribed': 1},
 {'open': 311, 'unopened': 922, 'bounced': 38, 'unsubscribed': 2},
 {'open': 429, 'unopened': 806, 'bounced': 39, 'unsubscribed': 0},
 {'open': 436, 'unopened': 799, 'bounced': 63, 'unsubscribed': 2},
 {'open': 387, 'unopened': 877, 'bounced': 48, 'unsubscribed': 0},
 {'open': 444, 'unopened': 832, 'bounced': 47, 'unsubscribed': 0},
 {'open': 443, 'unopened': 842, 'bounced': 63, 'unsubscribed':

In [13]:
if vbout.check_list_lengths(*lists):
    campaign_stats_data = vbout.merge_lists_dict(campaign_data_list, campaigns_stats)
    campaign_stats_data = vbout.add_to_list(campaign_stats_data, campaign_titles, label='title')
    campaign_stats_data = vbout.add_to_list(campaign_stats_data, total_all_clicks_stats, label='total_clicks')
    campaign_stats_data = vbout.add_to_list(campaign_stats_data, total_unique_clicks_stats, label='total_clicks_unique')

In [14]:
campaign_stats_data

[{'id': '203942',
  'creation_date': '03/20/2024 12:57',
  'subject': "4 solutions à forte valeur ajoutée pour faire avancer l'industrie de la construction en Afrique.",
  'sent_date': '04/12/2024 10:47',
  'year': '2024',
  'month': '4',
  'day_month': '12',
  'delivery_success': '1157',
  'delivery_failed': '30',
  'from_name': 'SIPPEC',
  'open': 245,
  'unopened': 912,
  'bounced': 30,
  'unsubscribed': 0,
  'title': 'NL Avril 2024',
  'total_clicks': 48,
  'total_clicks_unique': 17},
 {'id': '196728',
  'creation_date': '02/01/2024 18:11',
  'subject': "4 solutions à forte valeur ajoutée pour faire avancer l'industrie de la construction en Afrique.",
  'sent_date': '02/02/2024 10:39',
  'year': '2024',
  'month': '2',
  'day_month': '2',
  'delivery_success': '1167',
  'delivery_failed': '28',
  'from_name': 'SIPPEC',
  'open': 421,
  'unopened': 746,
  'bounced': 28,
  'unsubscribed': 0,
  'title': 'NL Février 2024',
  'total_clicks': 79,
  'total_clicks_unique': 29},
 {'id': '18

In [15]:
df = vbout.create_dataframe(campaign_stats_data)

In [16]:
df.head()

,id,creation_date,subject,sent_date,year,month,day_month,delivery_success,delivery_failed,from_name,open,unopened,bounced,unsubscribed,title,total_clicks,total_clicks_unique,total_deliveries
0,203942,03/20/2024 12:57,4 solutions à forte valeur ajoutée pour faire ...,04/12/2024 10:47,2024,4,12,1157,30,SIPPEC,245,912,30,0,NL Avril 2024,48,17,1187
1,196728,02/01/2024 18:11,4 solutions à forte valeur ajoutée pour faire ...,02/02/2024 10:39,2024,2,2,1167,28,SIPPEC,421,746,28,0,NL Février 2024,79,29,1195
2,189998,12/30/2023 11:13,4 solutions à forte valeur ajoutée pour faire ...,12/30/2023 11:21,2023,12,30,1175,27,SIPPEC,238,937,27,1,NL voeux 2024,8,8,1202
3,186862,12/09/2023 06:32,4 solutions à forte valeur ajoutée pour faire ...,12/09/2023 06:59,2023,12,9,1182,30,SIPPEC,235,947,30,1,NL Décembre 2023,29,14,1212
4,181244,11/14/2023 05:49,4 solutions à forte valeur ajoutée pour faire ...,11/16/2023 08:04,2023,11,16,1191,31,SIPPEC,223,968,31,1,NL Novembre 23,43,9,1222


In [1]:
import pandas as pd
df = pd.read_csv('resultats/Data/SIPPEC Campagnes Mailing.csv')

In [2]:
df.head()

,id,creation_date,subject,sent_date,year,month,day_month,delivery_success,delivery_failed,from_name,open,unopened,bounced,unsubscribed,title,total_clicks,total_clicks_unique,total_deliveries
0,203942,03/20/2024 12:57,4 solutions à forte valeur ajoutée pour faire ...,04/12/2024 10:47,2024,4,12,1157,30,SIPPEC,245,912,30,0,NL Avril 2024,48,17,1187
1,196728,02/01/2024 18:11,4 solutions à forte valeur ajoutée pour faire ...,02/02/2024 10:39,2024,2,2,1167,28,SIPPEC,421,746,28,0,NL Février 2024,79,29,1195
2,189998,12/30/2023 11:13,4 solutions à forte valeur ajoutée pour faire ...,12/30/2023 11:21,2023,12,30,1175,27,SIPPEC,238,937,27,1,NL voeux 2024,8,8,1202
3,186862,12/09/2023 06:32,4 solutions à forte valeur ajoutée pour faire ...,12/09/2023 06:59,2023,12,9,1182,30,SIPPEC,235,947,30,1,NL Décembre 2023,29,14,1212
4,181244,11/14/2023 05:49,4 solutions à forte valeur ajoutée pour faire ...,11/16/2023 08:04,2023,11,16,1191,31,SIPPEC,223,968,31,1,NL Novembre 23,43,9,1222


In [2]:
df.shape

(33, 18)

In [3]:
from spreadsheets import GoogleSheetsService
from credential.secrets import sippec, path

In [4]:
sheet_service = GoogleSheetsService(path.get('CREDENTIALS_PATH'), sippec.get('SPREADSHEET_NAME'))

Connecté à la feuille de calcul : SIPPEC Reporting Mailing Data


In [6]:
df.columns

Index(['id', 'creation_date', 'subject', 'sent_date', 'year', 'month',
       'day_month', 'delivery_success', 'delivery_failed', 'from_name', 'open',
       'unopened', 'bounced', 'unsubscribed', 'title', 'total_clicks',
       'total_clicks_unique', 'total_deliveries'],
      dtype='object')

In [7]:
aggregation_fields = ['delivery_success', 'delivery_failed', 'unsubscribed', 'open', 'unopened', 'bounced', 'total_clicks', 'total_clicks_unique', 'total_deliveries']

In [5]:
aggregated_df = sheet_service.aggregated_stats_by_month(df)
aggregated_df.head()

,year,month,delivery_success,delivery_failed,unsubscribed,open,unopened,bounced,total_clicks,total_clicks_unique,total_deliveries,from_name
21,2024,4,1157,30,0,245,912,30,48,17,1187,SIPPEC
20,2024,2,1167,28,0,421,746,28,79,29,1195,SIPPEC
19,2023,12,2357,57,2,473,1884,57,37,22,2414,SIPPEC
18,2023,11,1191,31,1,223,968,31,43,9,1222,SIPPEC
17,2023,10,1204,24,1,244,960,24,30,12,1228,SIPPEC


In [6]:
aggregated_df.shape

(22, 12)

In [7]:
sheet_service.save_flushed_sheet(dataframe=df)

Feuille 'Rapport_Mailing_Mensuel_SIPPEC_4_2024' introuvable.
Feuille 'Rapport_Mailing_Mensuel_SIPPEC_4_2024' créée avec succès.
DataFrame écrit avec succès dans la feuille de calcul.
Données agrégées pour le 4-2024 écrites dans la feuille : Rapport_Mailing_Mensuel_SIPPEC_4_2024
Feuille 'Rapport_Mailing_Mensuel_SIPPEC_2_2024' introuvable.
Feuille 'Rapport_Mailing_Mensuel_SIPPEC_2_2024' créée avec succès.
DataFrame écrit avec succès dans la feuille de calcul.
Données agrégées pour le 2-2024 écrites dans la feuille : Rapport_Mailing_Mensuel_SIPPEC_2_2024
Feuille 'Rapport_Mailing_Mensuel_SIPPEC_12_2023' introuvable.
Feuille 'Rapport_Mailing_Mensuel_SIPPEC_12_2023' créée avec succès.
DataFrame écrit avec succès dans la feuille de calcul.
Données agrégées pour le 12-2023 écrites dans la feuille : Rapport_Mailing_Mensuel_SIPPEC_12_2023
Feuille 'Rapport_Mailing_Mensuel_SIPPEC_11_2023' introuvable.
Feuille 'Rapport_Mailing_Mensuel_SIPPEC_11_2023' créée avec succès.
DataFrame écrit avec succès d

In [22]:
aggregated_df.head()

,year,month,delivery_success,delivery_failed,unsubscribed,open,unopened,bounced,total_clicks,total_clicks_unique,total_deliveries,from_name
21,2024,4,1157,30,0,245,912,30,48,17,1187,SIPPEC
20,2024,2,1167,28,0,421,746,28,79,29,1195,SIPPEC
19,2023,12,2357,57,2,473,1884,57,37,22,2414,SIPPEC
18,2023,11,1191,31,1,223,968,31,43,9,1222,SIPPEC
17,2023,10,1204,24,1,244,960,24,30,12,1228,SIPPEC


In [20]:
dataframe_pivot = sheet_service.get_dataframe_pivot(aggregated_df[:1])